<a href="https://colab.research.google.com/github/snipaid-nlg/models/blob/main/news-headline-generation/news-headline-generation-with-mt0-base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Headline Generation with mt0-base
Evaluating mt0-base on the task of generating headlines from journalistic text.

## Installs and Imports

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Dataset
from transformers.pipelines.pt_utils import KeyDataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import pandas as pd
from tqdm.auto import tqdm

## Loading and transforming 10kGNAD
We load the [Ten Thousand German News Articles Dataset](https://tblock.github.io/10kGNAD/) from Huggingface. The first sentence of a text (up to the first period) is a headline, so we split the text and use the second part. A Prefix `summarize:` is added because the model was trained on the task of summarization.

In [ ]:
gnad10 = load_dataset("gnad10")
df = pd.DataFrame(gnad10["train"])
df[['headline', 'fulltext']] = df["text"].str.split('.', n=1, expand=True)
df.drop(['label','text'], axis=1, inplace=True)
df['fulltext'] = "summarize: " + df['fulltext']
df.head()

  0%|          | 0/2 [00:00<?, ?it/s]

,headline,fulltext
0,21-Jähriger fällt wohl bis Saisonende aus,summarize: Wien – Rapid muss wohl bis Saisone...
1,"Erfundene Bilder zu Filmen, die als verloren g...",summarize: Wien – Die Filmgeschichte ist ein ...
2,Der frischgekürte CEO Sundar Pichai setzt auf ...,summarize: Die Atmosphäre im Silicon Valley i...
3,"Putin: ""Einigung, dass wir Menge auf Niveau vo...",summarize: Moskau – Die russischen Ölproduzen...
4,Estland sieht den künftigen österreichischen P...,summarize: Wien/Tallinn/Pskow – Die Eltern de...


Converting the pandas dataframe to a Huggingface dataset object

In [ ]:
dataset = Dataset.from_pandas(df)

## Loading the model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-base", use_cache=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("bigscience/mt0-base")

In [ ]:
template = {
    "num_beams": 5,
    "length_penalty": 0.5,
    "min_new_tokens": 10,
    "no_repeat_ngram_size": 2,
    "repetition_penalty": 2.0,
    "diversity_penalty": 0.0,
    "num_beam_groups": 1,
    "do_sample": False,
    "top_k": 50,
    "top_p": 0.9,
    "temperature": 1.0,
    "early_stopping": False,
    "num_return_sequences": 1
}

In [ ]:
pipe = pipeline("text2text-generation",
                device=0,
                model=model, 
                tokenizer=tokenizer, 
                **template)

In [ ]:
for out in tqdm(pipe(KeyDataset(dataset, "fulltext"))):
    print(out)

  0%|          | 0/9245 [00:00<?, ?it/s]

[{'generated_text': 'Admira Wacker Mödling (Rapid) skipper Thomas Murg remains'}]
[{'generated_text': 'The Forbidden Room, ein Two-Reeler'}]
[{'generated_text': 'Silicon Valley is rau. Among those in the top firms are brutale Um'}]
[{'generated_text': 'Russland says, a massive Förderung im In- und Ausland s'}]
[{'generated_text': 'Sascha Van der Bellen, künftiger Bundespräsident, stunned estn'}]
[{'generated_text': 'Trodat Trotec Holding GmbH übernimmt zwei US-Unternehmen, im Laserbereich '}]
[{'generated_text': 'Heracles Almelo stunned FC Utrecht in their Europa League qualifier'}]
[{'generated_text': 'ORF-Stiftungsrat außerordentlich Arbeitssitzung am Küniglberg,'}]
[{'generated_text': 'Militant Pakistan says 43 extremists have been killed.'}]
[{'generated_text': 'Bayern Munich skipper Robert Lewandowski will triumphen in allianz'}]
[{'generated_text': 'Domenico Berardi struck Juventus 1-0 in Serie A.'}]


KeyboardInterrupt: ignored